In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
pd.options.mode.chained_assignment = None

In [2]:
nbadata = pd.read_csv("game.csv")
nbadata.dtypes

season_id                   int64
team_id_home                int64
team_abbreviation_home     object
team_name_home             object
game_id                     int64
game_date                  object
matchup_home               object
wl_home                    object
min                         int64
fgm_home                  float64
fga_home                  float64
fg_pct_home               float64
fg3m_home                 float64
fg3a_home                 float64
fg3_pct_home              float64
ftm_home                  float64
fta_home                  float64
ft_pct_home               float64
oreb_home                 float64
dreb_home                 float64
reb_home                  float64
ast_home                  float64
stl_home                  float64
blk_home                  float64
tov_home                  float64
pf_home                   float64
pts_home                    int64
plus_minus_home             int64
video_available_home        int64
team_id_away  

In [20]:
## filter out data before 1976
nbadata['game_date'] = pd.to_datetime(nbadata['game_date']).dt.floor('D')
nbadata = nbadata[nbadata['game_date'].dt.year >= 1976]
nbadata = nbadata.reset_index(drop=True)

## filter out necessary column for calculating elo
need_column = ['season_id','game_id', 'game_date',
               'team_abbreviation_home','wl_home','plus_minus_home',
               'wl_away','team_abbreviation_away',
               'team_name_home','team_name_away']

nbadata = nbadata[need_column]
# data set used for test
# small_nbadata = nbadata.iloc[:500,:].reset_index(drop=True)

teams_abb = []
for team_abb in list(nbadata['team_abbreviation_home']):
    if team_abb not in teams_abb:
        teams_abb.append(team_abb)

teams_full = []
for team_abb in teams_abb:
    for game in range(0, nbadata.shape[0]):
        if team_abb == nbadata['team_abbreviation_home'][game]:
            teams_full.append(nbadata['team_name_home'][game])
            break

## Create original elo table
elo_table = pd.DataFrame({col: [1500] for col in teams_abb})
elo_table.insert(0, column=nbadata.columns[0], value=None)
elo_table.insert(1, column=nbadata.columns[1], value=None)
elo_table.insert(2, column=nbadata.columns[2], value=None)
elo_table

,season_id,game_id,game_date,POR,WAS,SEA,DET,CHI,BUF,PHL,...,PHI,UTA,SAS,MEM,NOH,CHA,NOK,OKC,BKN,NOP
0,None,None,None,1500,1500,1500,1500,1500,1500,1500,...,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500


In [6]:
def update_elo_match(home_elo, away_elo, MOV):
    E_home = 1/(1+pow(10,(away_elo - home_elo)/400))
    E_away = 1/(1+pow(10,(home_elo - away_elo)/400))

    if MOV > 0:## home win
        k = 20*(pow(MOV+3,0.8)/(7.5+0.006*(home_elo - away_elo)))
        
        updated_home_elo = k * (1 - E_home) + home_elo
        updated_away_elo = k * (0 - E_away) + away_elo

        return updated_home_elo,updated_away_elo

    else: ## away win
        k = 20*(pow(-MOV+3,0.8)/(7.5+0.006*(away_elo - home_elo)))

        updated_home_elo = k * (0 - E_home) + home_elo
        updated_away_elo = k * (1 - E_away) + away_elo

        return updated_home_elo,updated_away_elo

In [21]:
for a_game in range(0,len(nbadata)):
    ## store the new game information
    game = nbadata.loc[a_game,]
    home_team_name = game['team_abbreviation_home']
    away_team_name = game['team_abbreviation_away']
    MOV = game['plus_minus_home']

    ## store the previous elo information
    new_row_elo = elo_table.loc[a_game,]
    home_elo = new_row_elo[home_team_name]
    away_elo = new_row_elo[away_team_name]
    updated_home_elo,updated_away_elo = update_elo_match(home_elo, away_elo, MOV)

    ## update the game and season information in the new row
    for game_information_index in range(0,3):
        new_row_elo[game_information_index] = game[game_information_index]
    
    ## update elo for two teams after the game
    new_row_elo[home_team_name] = updated_home_elo
    new_row_elo[away_team_name] = updated_away_elo

    ## update elo for all teams after a season
    if new_row_elo['season_id'] != elo_table.loc[elo_table.shape[0]-1,'season_id'] and elo_table.shape[0] > 1:
        new_row_elo[3:len(new_row_elo)] *= 0.75
        new_row_elo[3:len(new_row_elo)] += 0.25*1505
    
    ## reset those who do not participate in to the season
    new_row_elo = np.where(new_row_elo == 1501.25, 1500, new_row_elo)

    ## append the new row to the elo table
    elo_table.loc[elo_table.shape[0]] = new_row_elo



In [25]:
grouped_elo = elo_table.groupby('game_date')

aggregations = {
    'season_id': 'last',
    'game_id': 'last',
    'game_date': 'last'
}

for team in teams_abb:
    aggregations[team] = 'last'

long_elo_table = grouped_elo.agg(aggregations).reset_index(drop=True)
long_elo_table

,season_id,game_id,game_date,POR,WAS,SEA,DET,CHI,BUF,PHL,...,PHI,UTA,SAS,MEM,NOH,CHA,NOK,OKC,BKN,NOP
0,21975,27500289,1976-01-01,1494.409383,1511.011365,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1,21975,27500293,1976-01-02,1494.409383,1519.726249,1493.675631,1486.536538,1491.285116,1504.041911,1511.011365,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
2,21975,27500304,1976-01-03,1516.127693,1532.761193,1493.675631,1492.918060,1491.285116,1499.879052,1497.976421,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
3,21975,27500307,1976-01-04,1504.686907,1532.761193,1506.908521,1492.918060,1491.285116,1499.879052,1497.976421,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
4,21975,27500308,1976-01-05,1504.686907,1532.761193,1506.908521,1492.918060,1495.263102,1499.879052,1497.976421,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7145,22022,22200991,2023-03-08,1429.169478,1508.066474,1501.673466,1319.154662,1527.300064,1504.999438,1504.878356,...,1629.018564,1443.909205,1280.293821,1581.744782,1498.666952,1406.260347,1505.087676,1516.337539,1513.491451,1469.217792
7146,22022,22200994,2023-03-09,1429.169478,1508.066474,1501.673466,1311.835737,1527.300064,1504.999438,1504.878356,...,1629.018564,1452.873550,1280.293821,1596.238011,1498.666952,1413.579272,1505.087676,1516.337539,1509.654236,1469.217792
7147,22022,22201003,2023-03-10,1427.493920,1499.880626,1501.673466,1311.835737,1527.300064,1504.999438,1504.878356,...,1630.694122,1452.873550,1302.070005,1596.238011,1498.666952,1413.579272,1505.087676,1516.337539,1513.842625,1469.217792
7148,22022,22201013,2023-03-11,1427.493920,1499.880626,1501.673466,1306.973178,1530.327980,1504.999438,1504.878356,...,1630.694122,1460.684814,1302.070005,1600.601692,1498.666952,1405.768008,1505.087676,1527.061279,1513.842625,1458.494051


In [26]:
grouped_elo = long_elo_table.groupby('season_id')

aggregations = {
    'season_id': 'last',
}

for team in teams_abb:
    aggregations[team] = 'mean'

long_elo_table = grouped_elo.agg(aggregations).reset_index(drop=True)
long_elo_table

,season_id,POR,WAS,SEA,DET,CHI,BUF,PHL,NOJ,LAL,...,PHI,UTA,SAS,MEM,NOH,CHA,NOK,OKC,BKN,NOP
0,21975,1495.706165,1556.208429,1498.459176,1457.799825,1476.761374,1516.417560,1492.289290,1509.891317,1474.239545,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1,21976,1566.573474,1538.183203,1498.480269,1508.481244,1469.470389,1442.946434,1564.384825,1459.108346,1562.138336,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
2,21977,1658.354790,1537.980534,1517.959152,1483.311338,1532.814448,1423.392380,1627.448489,1454.188044,1550.241131,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
3,21978,1522.912667,1606.335667,1588.850405,1447.890880,1427.924792,1405.734300,1570.780422,1414.051066,1596.286390,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
4,21979,1535.337863,1480.757016,1623.230361,1378.232982,1434.670949,1430.550725,1604.709314,1401.037225,1620.801567,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
5,21980,1505.107119,1505.726962,1516.907782,1327.212149,1519.156897,1449.163044,1679.254055,1427.027919,1635.430716,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
6,21981,1530.956314,1531.080996,1564.353754,1481.019800,1499.268135,1463.122283,1658.106093,1446.520939,1616.317842,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
7,21982,1540.275143,1514.525962,1570.591512,1502.356981,1429.895014,1473.591712,1720.249858,1461.140704,1651.475360,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
8,21983,1575.906956,1489.071284,1516.133774,1559.864447,1420.745695,1481.443784,1607.665721,1472.105528,1596.867054,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
9,21984,1534.196111,1501.355446,1452.592677,1572.288934,1463.836461,1487.332838,1635.402431,1480.329146,1629.640932,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000


In [27]:
long_elo_table = long_elo_table.melt(id_vars='season_id', var_name='team', value_name='elo_rating')
long_elo_table


,season_id,team,elo_rating
0,21975,POR,1495.706165
1,21976,POR,1566.573474
2,21977,POR,1658.354790
3,21978,POR,1522.912667
4,21979,POR,1535.337863
...,...,...,...
2065,22018,NOP,1501.017112
2066,22019,NOP,1472.083621
2067,22020,NOP,1504.096546
2068,22021,NOP,1474.764248


In [30]:
## export the elo table
current_directory = os.getcwd()
csv_path = os.path.join(current_directory, 'long_elo_table.csv')
long_elo_table.to_csv(csv_path, index=False)
